In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from os import listdir
from ScanImageTiffReader import ScanImageTiffReader
from os.path import isfile, join
import h5py
import pandas as pd
from scipy import sparse
import caiman as cm
import json
from caiman.utils.visualization import get_contours
import tifffile as tf

In [2]:
directory = '/media/david/hdd1/David_GCAMP/20220119'
Select_ROI = {2:[19,24,52,72,81],3:[14,40,63,64],6:[16,24,47],7:[7],9:[23],10:[10,16],11:[16,25,28,29],5:[8,11,13,15,21,22,23,24,25,26,28,29,35,36,39,41,45,54,64,74,75,83,85,86,88,91,94,100,105,106]}
num_file = [2,3,6,7,10,11]
if not os.path.isdir(directory+'/movie'):
    os.mkdir(directory+'/movie')
num_slice = 1

In [3]:
for num in num_file:
    interest_ROIs = Select_ROI[num]
    filename = 'file_0000'+str(num)+'.tif'
    #Read tag file
    dict_name = directory+'/tag/'+filename[:-4]+'.json'
    with open(dict_name, 'r') as f:
        tag = json.load(f)
    frameStimulus = np.array(tag['frameStimulus'])
    frameOmitStimulus = np.array(tag['frameOmitStimulus'])
    frameTimestamps = np.array(tag['frameTimestamps'])
    #Read results
    name = directory+'/preprocess/'+filename[:-4]+'_'+str(num_slice)+'.hdf5'
    cnm2 = h5py.File(name,'r')
    shape = cnm2['estimates']['A']['shape'][:]
    indices = cnm2['estimates']['A']['indices'][:]
    indptr = cnm2['estimates']['A']['indptr'][:]
    data = cnm2['estimates']['A']['data'][:]
    dims = cnm2['dims'][:]
    A = sparse.csr_matrix((data, indices, indptr),shape=(shape[1],shape[0])).transpose()
    coordinates = get_contours(A, dims, thr=0.9)
    #movie = ScanImageTiffReader(directory+'/gaussian/'+filename).data()
    #movie = tf.imread(directory+'/gaussian/'+filename)
    movie = np.load(directory+'/denoise/denoise_'+filename[:-4]+'_1.npy')
    #Create movie
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    for interest_ROI in interest_ROIs:
        #out = cv2.VideoWriter(directory+'/gaussian_movie/'+filename[:-4]+'_No'+str(interest_ROI)+'.avi', fourcc, 30.0, (movie.shape[2],movie.shape[1]),0)
        out = cv2.VideoWriter(directory+'/movie/'+filename[:-4]+'_No'+str(interest_ROI)+'.avi', fourcc, 30.0, (movie.shape[2],movie.shape[1]),0)
        for i in range(movie.shape[0]):
            #Transform from int16 to uint8
            img = cv2.normalize(movie[i,:,:].reshape(movie.shape[2],movie.shape[1]), None, 0, 255, cv2.NORM_MINMAX)

            coordinate = coordinates[interest_ROI-1]['coordinates']
            for c in coordinate[1:-1]:
                if np.isnan(c[0]) or np.isnan(c[1]):
                    pass
                else:
                    if i < 100:
                        img[int(c[1]),int(c[0])] = 255
                    else:
                        img[int(c[1]),int(c[0])] = 128
            if frameStimulus[0,i] >1:
                img[0:10,0:10] = 255
                img[502:511,502:511] = 255
                img[0:10,502:511] = 255
                img[502:511,0:10] = 255
            elif frameOmitStimulus[0,i]> 1:
                img[250:260,0:10] = 255
                img[502:511,250:260] = 255
                img[0:10,250:260] = 255
                img[250:260,502:511] = 255
            out.write(np.uint8(img))
        out.release()

FileNotFoundError: [Errno 2] No such file or directory: '/media/david/hdd1/David_GCAMP/20220119/tag/file_000010.json'